# #1
문제: _서포트 벡터 머신 회귀(`sklearn.svm.SVR`)를 `kernel=“linear”`(하이퍼파라미터 `C`를 바꿔가며)나 `kernel=“rbf”`(하이퍼파라미터 `C`와 `gamma`를 바꿔가며) 등의 다양한 하이퍼파라미터 설정으로 시도해보세요. 서포트 벡터 머신은 대용량 데이터셋에 적용하기가 쉽지 않습니다. 따라서 훈련 세트의 처음 5,000개 샘플만 사용해 모델을 훈련하고 3-겹 교차 검증을 사용하세요. 그렇지 않으면 몇 시간이 걸릴 것입니다. 지금은 이 하이퍼파라미터가 무엇을 의미하는지 너무 신경 쓰지 마세요(궁금하다면 5장 노트북을 참고하세요). 최상의 `SVR` 모델은 무엇인가요?_

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

param_grid = [
    {
        "svr_kernel" : ["linear"],
        "svr_C" : [10.0,30.0,100.0,300.0,1000.0,3000.0,10000.0,30000.0],
    },
    {
        "svr_kernel" : ["linear"],
        "svr_C" : [1.0,3.0,10.0,30.0,100.0,300.0,1000.0],
        "svr_gamma": [0.01,0.03,0.1,0.3,1.0,3.0]

    },


]

svr_pipeline = Pipeline(
    [("preprocessing",preprocessing),("svr",SVR())]
)
grid_search = GridSearchCV(
    svr_pipeline, param_grid, cv=3, scoring = "neg_root_mean_squared_error"
)
grid_search.fit(housing.iloc[:5000], housing_label.iloc[:5000])

# #2
문제: _`GridSearchCV`를 `RandomizedSearchCV`로 바꿔보세요._

In [1]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import expon, loguniform

param_distribs = {
    "svr_kernel" : ["linear,"rbf],
    "svr_c":
}

rnd_search = RandomizedSearchCV(
    svr_pipeline,
    param_distributions=param_distribs,
    n_iter=50,
    cv=3
    scoring="neg_root_mean_squared_error",
    random_state=12345
    )
rnd_search.fit(housing.iloc[:5000],housing_labels.iloc[:5000])

# #3
질문: _가장 중요한 특성을 선택하는 `SelectFromModel` 변환기를 준비 파이프라인에 추가해보세요._

In [ ]:
selector_pipeline = Pipeline(
    [
        (
            "preprocessing":preprocessing
        ),
        (
            "selector": SelectFromModel(randomforestRegressor(random_srate=42),threshold = 0.005),
        ),
        (
            SVR(
                C=rnd_search.best_params_["svr_C"]
            )
        )
    ]
)

# #4
문제: _`fit()` 메서드 안에서 k-최근접 이웃 회귀(`sklearn.neighbors.KNeighborsRegressor`)를 훈련하고 `transform()` 메서드에서 이 모델의 예측을 반환하는 사용자 정의 변환기를 만들어 보세요. 이 변환기의 입력으로 위도와 경도를 사용하고 예측 결과를 하나의 특성으로 전처리 파이프라인에 추가하세요. 이렇게 하면 가장 가까운 구역의 중간 주택 가격에 대한 특성이 모델에 추가됩니다._

In [ ]:
class featurefromregressor(metaestimatorMixin, baseestimator,transformerMixin):
    def __init__(self, estimator):
        self.estimator = estimator
    
    def fit(self, X,y=None):
        estimator_ = clone(self.estimator)
        estimator_.fit(X,y)
        self.estimator_ = estimator_
        self.n_features_in = self.estimator_.n_features_in_
        if hasattr(self.estimaor, "feature_names_in_"):
            self.feature_names_in_ = self.estimator.feature_names_in_
        return self

    def transform(self,X):
        check_is_fitted(self)
        predictions = self.estimator_predict(X)
        if predictions.nidm == 1:
            predictions = predictions.reshape(-1,1)
        return predictions

    def get_feature_names_out(self, names= None):
        check_is_fitted(self)
        n_outputs = getattr(self.estimator_, "n_outputs_", 1)
        estimator_class_name = self.estimator_.__class__.__name__
        estimator_short_name = estimator_class_name.lower().replace("_", "")
        return [f"{estimator_short_name}_prediction_{i}" for i in range(n_outputs)]


# #5
문제: _`GridSearchCV`를 사용해 준비 단계의 옵션을 자동으로 탐색해보세요._

In [ ]:
param_distribs = {
    "preprocessing_geo_estimator_n_neighbors" : range(1,30),
    "preprocessing_geo_estimator_weights":["distance","uniform"],
    "svr_C" = loguniform(20,200_000),
    "svr_gamma" : expon(scale = 1.0),

}

new_geo_rnd_search = RandomizedSearchCV(
    new_geo_pipeline,
    param_distributions=param_distribs,
    n_iter = 50,
    cv= 3,
    scoring="neg_root_mean_s  "
)